# Migration Guide

This jupyter notebook demonstrates how to adapt a [pycycling](https://github.com/zacharyedwardbull/pycycling) device into Bleak-FSM.

For this example we will use . Sterzo is a bluetooth steering angle sensor that goes under the front wheel of a bicycle mounted on an indoor trainer. 

The implementation of Sterzo is at [pycycling/sterzo.py](https://github.com/zacharyedwardbull/pycycling/blob/master/pycycling/sterzo.py) but we only need to look at [examples/sterzo_examples.py](https://github.com/zacharyedwardbull/pycycling/blob/master/examples/sterzo_example.py).

For your convenience, here is `sterzo_example.py`:

```python

import asyncio
from bleak import BleakClient

from pycycling.sterzo import Sterzo


async def run(address):
    async with BleakClient(address) as client:
        def steering_handler(steering_angle):
            print(steering_angle)

        await client.is_connected()
        sterzo = Sterzo(client)
        sterzo.set_steering_measurement_callback(steering_handler)
        await sterzo.enable_steering_measurement_notifications()
        await asyncio.sleep(60)

if __name__ == "__main__":
    import os

    os.environ["PYTHONASYNCIODEBUG"] = str(1)

    device_address = "36A444C9-2A18-4B6B-B671-E0A8D3DADB1D"
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run(device_address))

```

To begin with, we'll import the same modules plus Bleak-FSM

In [1]:
import asyncio
from pycycling.sterzo import Sterzo

from bleak_fsm import machine, BleakModel

In [2]:
model = BleakModel() # This represents a given bluetooth adapter. Let's assume there's one on the system.
machine.add_model(model) # Add the model to the state machine

By default, `model.wrap` is a function that does nothing and just returns the input.
That's fine if we want to use the `client`, which is an instance of `BleakClient` directly.

However, we want to take advantage of the work that went into creating the `Sterzo` implementation.
Therefore we need to modify the `model.wrap` function as follows:

In [3]:
model.wrap = lambda client: Sterzo(client) # Wrap the model with the Sterzo class

If you look at [line 54 of sterzo.py](https://github.com/zacharyedwardbull/pycycling/blob/a5998eb191edb1b2f2591d26b06f442a4f1ccff3/pycycling/sterzo.py#L54), you can see that the steering measurement callback is called with one positional argument, called `steering_value`. 
The steering measurement callback is set according through `set_steering_measurement_callback()` ([line 46-47](https://github.com/zacharyedwardbull/pycycling/blob/a5998eb191edb1b2f2591d26b06f442a4f1ccff3/pycycling/sterzo.py#L46-L47))

We are responsible for creating and passing in a callback function that will become `self._steering_measurement_callback`

In [4]:
def handle_sterzo_measurement(value):
    print(f"Sterzo angle: {value}")

In [5]:
model.set_measurement_handler = lambda client: client.set_steering_measurement_callback(handle_sterzo_measurement)

Next, we set the `enable_notifications` and `disable_notifications` functions to their corresponding functions in [examples/sterzo_examples.py](https://github.com/zacharyedwardbull/pycycling/blob/master/examples/sterzo_example.py)

In [6]:
model.enable_notifications = lambda client: client.enable_steering_measurement_notifications()
model.disable_notifications = lambda client: client.disable_steering_measurement_notifications()

In [7]:
model.state

'Init'

In [8]:
asyncio.create_task(model.start_scan()) 
# This async function will wait for the stop_scan() before returning,
# So we don't `await` it. Instead we create a task and let it run in the background.

await asyncio.sleep(3)
await model.stop_scan()

True

In [9]:
model.bt_devices

{'0B017D91-48E0-8A7F-A878-F0F2F61A16E3': BLEDevice(0B017D91-48E0-8A7F-A878-F0F2F61A16E3, ProPods Aero),
 '6FE76CFE-0681-FC03-E66A-C925EDB230F1': BLEDevice(6FE76CFE-0681-FC03-E66A-C925EDB230F1, None),
 '14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
 '37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4': BLEDevice(37DA5EA4-DB46-2D6D-9E8F-A7B7D4DBCCA4, GiGA Genie 3_F70E),
 '715D8603-DC4C-2994-C0CD-2BC5A93E0B38': BLEDevice(715D8603-DC4C-2994-C0CD-2BC5A93E0B38, STERZO),
 'DC2343A7-C46E-E1BB-30E9-BC6E274B4A39': BLEDevice(DC2343A7-C46E-E1BB-30E9-BC6E274B4A39, Fast Fred),
 '20BF6293-9322-9334-F51A-F51C102FD12F': BLEDevice(20BF6293-9322-9334-F51A-F51C102FD12F, None),
 '9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41': BLEDevice(9E41F52B-BDEF-3C85-F51D-E0DE23B7CB41, Study),
 '3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862': BLEDevice(3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862, Green Apple),
 '7ADF6295-3580-8840-4B61-5A84E1ABCD6E': BLEDevice(7ADF6295-3580-8840-4B61-5A84E1ABCD6E, None)

In [10]:
await model.set_target("715D8603-DC4C-2994-C0CD-2BC5A93E0B38")

True

In [11]:
await model.connect()

Connected to 715D8603-DC4C-2994-C0CD-2BC5A93E0B38


True

In [12]:
asyncio.create_task(model.stream())
# Like `model.start_scan()`, this async function will wait for `disconnect()` before returning,
# So we don't `await` it. Instead we create a task and let it run in the background.

<Task pending name='Task-13' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [13]:
await asyncio.sleep(10)

Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: nan
Sterzo angle: 5.1046142578125
Sterzo angle: 5.0430755615234375
Sterzo angle: 4.750457763671875
Sterzo angle: 4.750457763671875
Sterzo angle: 4.573921203613281
Sterzo angle: 4.985389709472656
Sterzo angle: 4.8101959228515625
Sterzo angle: 5.0467071533203125
Sterzo angle: 5.2822265625
Sterzo angle: 4.3977508544921875
Sterzo angle: 4.632926940917969
Sterzo angle: 5.339286804199219
Sterzo 

In [14]:
await model.disconnect()

Stopped streaming
Disconnected from 715D8603-DC4C-2994-C0CD-2BC5A93E0B38


True